# Исследовательский анализ профилей потребления пользователей интернет-магазина "Пока все еще тут"

## Материалы  
[презентация](https://drive.google.com/file/d/1ZQf3DY9k2iywRuIsTg3M1KeCNh9TBGXh/view?usp=sharing)

## Описание данных

Датасет описывает транзакции интернет магазина товаров для дома и быта "Пока все ещё тут"   

Колонки `ecom_dataset_upd.csv`:   
`date` - дата заказа   
`customer_id` - идентификатор покупателя   
`order_id` - идентификатор заказа   
`product` - наименование товара   
`quantity` - количество товара в заказе   
`price` - цена товара  



## Декомпозиция

 
### Цели: 
1. разработать рекомендации: каким клиентам и когда рассылать рекламные предложения о каких-либо категориях товаров   
2. разработать стратегию по увеличению количества продаваемого товара для каждого сегмента    
3. предложить способы повышения возвращаемости покупателей для различных сегментов



### Предобработка


1. форматирование названия столбцов
2. проверка типов данных
3. приведение данных к нужному типу
4. проверка на явные и неявные дубликаты 
5. устранение дубликатов 
6. проверка на пропуски
7. устранение пропусков
8. приведение даты к нужному формату и типу


### Исследовательский аналих данных


1. Узнать период за который собраны данные
2. найти выбросы и их обработать
3. выделить основные показатели интернет-магазина
    1) динамика выручки
    2) средний чек и его динамику
    3) помесячную динамику средней выручки
4. сегментация покупателей на основе из истории покупок при помощи RFM анализа
5. анализ сегментов
    1) количество покупателей
    2)  средний чек
    3) основные продукты и категории
    
6. анализ сезонности товаров в разрезе сегментов покупателей 

7. Гипотезы   
    1) Нулевая гипотеза: средние чеки сегментов групп равны   
    Альт. гипотеза: средние чеки сегментов групп разные
       
    2) Нулевая гипотеза: средняя выручка сегментов групп одинаковая     
    Альт. гипотеза: ср. выручка сегментов групп разная 


## Загрузка библиотек и файлов исследования

Выполним загрузку  для анализа и файлов исследования

In [79]:
import pandas as pd
import plotly.express as px


import numpy as np
import scipy.stats as st

# для вывода полного содержимого ячейки
pd.options.display.max_colwidth = 190


import warnings
warnings.filterwarnings('ignore')




data = pd.read_csv(
        'https://code.s3.yandex.net/datasets/ecom_dataset_upd.csv')

Так как состояние данных - это неизвестность пока их не посмотришь, выполним этап предобработки - ведь от их качества зависит все результатаы исследования

## Предобработка

 
Для начала отформатируем названия столбцов - избавимся от пробелов слева и/или справа от названия столбцов

In [80]:
data.columns = list(map(lambda x: x.strip(), data.columns))

Посмотрим на данные при помощи самодельной функции

In [81]:
def get_info_dataframe(dataframe,
                       transpose=False,
                       count_rows=5
                       ):
    """
    Получение основной информации о датафрейме\n

    `dataframe` - исследуемый датафрейм\n
    `transpose` - перевернуть таблицу на 90°\n
    `count_rows` - количество первых выводимых строк
    """

    # -----------------сборка таблицы с пропусками---------------------
    # число пропусков
    count_missing = dataframe.isna().sum()
    
    # доля пропусков
    missing_percent = count_missing / len(dataframe)*100

    # сборка
    df = pd.DataFrame(
        data={
            'missing_percent': missing_percent,
            'count_missing': count_missing
        })
    
    # цветной бар в столбце с долей пропусков и добапвляем знак %
    table = (df
             .style
             .bar(
                 subset="missing_percent",
                 vmax=100,
                 color='red'
             )
             .format(
                 subset="missing_percent",
                 formatter="{:.2f} %"
             )
             )
    # --------------------------------------

    # сводная информация
    print(dataframe.info())

    # часть таблицы
    if transpose == True:
        display(dataframe.head(count_rows).T)
    else:
        display(dataframe.head(count_rows))

    # число дубликатов
    print(
        f'\nявных дубликатов: {dataframe.duplicated().sum()}\n{". "*10}'.upper())

    # число пропусков
    print(f'\nпропуски'.upper())
    display(table)

    print('----'*20)

In [82]:
get_info_dataframe(data, count_rows=15)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7474 entries, 0 to 7473
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         7474 non-null   int64  
 1   customer_id  7474 non-null   object 
 2   order_id     7474 non-null   int64  
 3   product      7474 non-null   object 
 4   quantity     7474 non-null   int64  
 5   price        7474 non-null   float64
dtypes: float64(1), int64(3), object(2)
memory usage: 350.5+ KB
None


,date,customer_id,order_id,product,quantity,price
0,2018100100,ee47d746-6d2f-4d3c-9622-c31412542920,68477,"Комнатное растение в горшке Алое Вера, d12, h30",1,142.0
1,2018100100,ee47d746-6d2f-4d3c-9622-c31412542920,68477,"Комнатное растение в горшке Кофе Арабика, d12, h25",1,194.0
2,2018100100,ee47d746-6d2f-4d3c-9622-c31412542920,68477,Радермахера d-12 см h-20 см,1,112.0
3,2018100100,ee47d746-6d2f-4d3c-9622-c31412542920,68477,Хризолидокарпус Лутесценс d-9 см,1,179.0
4,2018100100,ee47d746-6d2f-4d3c-9622-c31412542920,68477,Циперус Зумула d-12 см h-25 см,1,112.0
5,2018100100,ee47d746-6d2f-4d3c-9622-c31412542920,68477,Шеффлера Лузеана d-9 см,1,164.0
6,2018100100,ee47d746-6d2f-4d3c-9622-c31412542920,68477,Юкка нитчатая d-12 см h-25-35 см,1,134.0
7,2018100108,375e0724-f033-4c76-b579-84969cf38ee2,68479,Настенная сушилка для белья Gimi Brio Super 100,1,824.0
8,2018100108,6644e5b4-9934-4863-9778-aaa125207701,68478,"Таз пластмассовый 21,0 л круглый ""Водолей"" С614, 1404056",1,269.0
9,2018100109,c971fb21-d54c-4134-938f-16b62ee86d3b,68480,Чехол для гладильной доски Colombo Persia Beige 130х50 см из хлопка 5379,1,674.0



ЯВНЫХ ДУБЛИКАТОВ: 0
. . . . . . . . . . 

ПРОПУСКИ


,missing_percent,count_missing
date,0.00 %,0
customer_id,0.00 %,0
order_id,0.00 %,0
product,0.00 %,0
quantity,0.00 %,0
price,0.00 %,0


--------------------------------------------------------------------------------


Явных дубликатов  и пропусков тоже нет во всем датафрейме.

Проверим на наличие и устраним неявные дубликаты- такие могут встретиться в столбце с товаром `product`

Для начала запомним сколько было уникальных товаров до предобработки

In [83]:
count_unique_product_old = data['product'].nunique()

Приведем все названия товара к нижнему регистру. Так как в названии товара где описаны его габариты в виде диаметра и высоты разнятся по оформлению, например `h-12` и `h12` то приведем их к единообразию убрав знак `-`. Буквы `ё` меняем на `е`, запятые тоже устраняем

In [84]:
data['product'] = (data['product']
                   .apply(lambda x: x
                          .strip()
                          .lower()
                          .replace('d-', 'd')
                          .replace('h-', 'h')
                          .replace(',', '')
                          .replace('ё', 'е')
                          )
                   )

посчитаем число уникальных товаров после обработки и выведем количество устаненных(никуда не делись просто стали как близнецы) неявных дубликатов

In [85]:
count_unique_product_new = data['product'].nunique()

print('Устранено неявных дубликатов:',
      count_unique_product_old - count_unique_product_new)

Устранено неявных дубликатов: 5


Проверим теперь на явные дубликаты в ключевых столбцах

посмоьтрим, есть ли такие номера заказов совершенные разными покупателями

In [86]:
duplicat_orders = (data
                   .groupby('order_id')
                   ['customer_id']
                   .nunique()
                   .reset_index()
                   .rename(
                       columns=dict(customer_id='times')
                   )
                   )



duplicat_orders.head()

,order_id,times
0,12624,1
1,13547,1
2,14480,1
3,14481,1
4,14482,1


и посчитаем сколько покупателей "владеют" чужими номерами заказов

In [87]:
(duplicat_orders
 .groupby('times')['order_id']
 .count()
 .reset_index()
 .rename(
     columns={'order_id': 'count_order_id'}
 )
 )

,times,count_order_id
0,1,3492
1,2,26
2,3,2
3,4,1


Получается, что существует 3492 уникальных номера заказа которые не повторяются у других пользователей, остальные- встречаются у 2,3 и 4 разлтчных пользователей N-раз. Такое следует исправить.

отфильтруем датасет по уникальным заказам взятыми из `duplicat_orders`

In [88]:
data = (data[
    data['order_id']
    .isin(
        duplicat_orders
        .query('times==1')['order_id'])
]
)

теперь проверим дубликаты по пользователю,номеру заказа и его содержимому. 

In [89]:
(data
 .duplicated(
     subset=[
         'customer_id',
         'order_id',
         'product',
         'quantity',
         'price'
     ]
 ).sum()
 )

1841

Они есть. Ликвидируем

In [90]:
data = data.drop_duplicates().reset_index(drop=True)

Теперь вернемся к типам данных и посмотрим на дату.
Во-первых это число , а во-вторых запись очень похожа на дату в виде `ГГГГММДДЧЧ`,  
где `ГГГГ` - год    
`ММ` - месяц     
`ДД` -день   
`ЧЧ` - час   

приведем ее к виду `ГГГГ-ММ-ДД ЧЧ:00:00`


In [91]:
data['datetime'] = pd.to_datetime(data['date'], format='%Y%m%d%H')

data.head(10)

,date,customer_id,order_id,product,quantity,price,datetime
0,2018100100,ee47d746-6d2f-4d3c-9622-c31412542920,68477,комнатное растение в горшке алое вера d12 h30,1,142.0,2018-10-01 00:00:00
1,2018100100,ee47d746-6d2f-4d3c-9622-c31412542920,68477,комнатное растение в горшке кофе арабика d12 h25,1,194.0,2018-10-01 00:00:00
2,2018100100,ee47d746-6d2f-4d3c-9622-c31412542920,68477,радермахера d12 см h20 см,1,112.0,2018-10-01 00:00:00
3,2018100100,ee47d746-6d2f-4d3c-9622-c31412542920,68477,хризолидокарпус лутесценс d9 см,1,179.0,2018-10-01 00:00:00
4,2018100100,ee47d746-6d2f-4d3c-9622-c31412542920,68477,циперус зумула d12 см h25 см,1,112.0,2018-10-01 00:00:00
5,2018100100,ee47d746-6d2f-4d3c-9622-c31412542920,68477,шеффлера лузеана d9 см,1,164.0,2018-10-01 00:00:00
6,2018100100,ee47d746-6d2f-4d3c-9622-c31412542920,68477,юкка нитчатая d12 см h25-35 см,1,134.0,2018-10-01 00:00:00
7,2018100108,375e0724-f033-4c76-b579-84969cf38ee2,68479,настенная сушилка для белья gimi brio super 100,1,824.0,2018-10-01 08:00:00
8,2018100108,6644e5b4-9934-4863-9778-aaa125207701,68478,"таз пластмассовый 210 л круглый ""водолей"" с614 1404056",1,269.0,2018-10-01 08:00:00
9,2018100109,c971fb21-d54c-4134-938f-16b62ee86d3b,68480,чехол для гладильной доски colombo persia beige 130х50 см из хлопка 5379,1,674.0,2018-10-01 09:00:00


так как нам определенно понадобится просто дата без времени, то создадим столбец в который запишем дату без времени. Но для удобства уберем старый столбец `date`

In [92]:
data.drop(columns='date', inplace=True)
data.columns

Index(['customer_id', 'order_id', 'product', 'quantity', 'price', 'datetime'], dtype='object')

создвеам столбец с датой

In [93]:
data['date'] = data['datetime'].dt.date
data.head(10)

,customer_id,order_id,product,quantity,price,datetime,date
0,ee47d746-6d2f-4d3c-9622-c31412542920,68477,комнатное растение в горшке алое вера d12 h30,1,142.0,2018-10-01 00:00:00,2018-10-01
1,ee47d746-6d2f-4d3c-9622-c31412542920,68477,комнатное растение в горшке кофе арабика d12 h25,1,194.0,2018-10-01 00:00:00,2018-10-01
2,ee47d746-6d2f-4d3c-9622-c31412542920,68477,радермахера d12 см h20 см,1,112.0,2018-10-01 00:00:00,2018-10-01
3,ee47d746-6d2f-4d3c-9622-c31412542920,68477,хризолидокарпус лутесценс d9 см,1,179.0,2018-10-01 00:00:00,2018-10-01
4,ee47d746-6d2f-4d3c-9622-c31412542920,68477,циперус зумула d12 см h25 см,1,112.0,2018-10-01 00:00:00,2018-10-01
5,ee47d746-6d2f-4d3c-9622-c31412542920,68477,шеффлера лузеана d9 см,1,164.0,2018-10-01 00:00:00,2018-10-01
6,ee47d746-6d2f-4d3c-9622-c31412542920,68477,юкка нитчатая d12 см h25-35 см,1,134.0,2018-10-01 00:00:00,2018-10-01
7,375e0724-f033-4c76-b579-84969cf38ee2,68479,настенная сушилка для белья gimi brio super 100,1,824.0,2018-10-01 08:00:00,2018-10-01
8,6644e5b4-9934-4863-9778-aaa125207701,68478,"таз пластмассовый 210 л круглый ""водолей"" с614 1404056",1,269.0,2018-10-01 08:00:00,2018-10-01
9,c971fb21-d54c-4134-938f-16b62ee86d3b,68480,чехол для гладильной доски colombo persia beige 130х50 см из хлопка 5379,1,674.0,2018-10-01 09:00:00,2018-10-01


Теперь данные готовы для дальнейшего анализа

## Исследовательский анализ данных

### Изучаемый период

Для начала узнаем период за который предоставленны данные

In [94]:
print(data['date'].min(), '-', data['date'].max())

2018-10-01 - 2020-01-31


в годах времени прошло мало, а вот в месяцах - больше 12. Отметим это для себя при построении графиков от времени

### Проверка на выбросы и их обработка

Поищем выбросы среди полной стоимости заказа и количества заказов на пользователя.   
Сначала нужно посчитать стоимость товарной позиции . Потом собрать таблицу с стоимость товаров в заказе

In [95]:
data['total_price'] = data['quantity'] * data['price']

In [96]:
info_chek = (data
             .groupby(['order_id'], as_index=False)
             .aggregate({'total_price': 'sum'})
             .rename(columns=dict(total_price='total_coast')))


info_chek.head()

,order_id,total_coast
0,12624,375.0
1,13547,4788.0
2,14480,359.0
3,14481,600.0
4,14482,376.0


смотрим сумму заказов

In [156]:
px.scatter(info_chek,
                 y='total_coast',
                 labels={'total_coast': 'сумма',
                         'index': 'заказ'},
                 marginal_y='rug',
                 title='Общая сумма заказа')



как видим есть выброс в 675 000 руб и весьма значительный. Устраним его и отфильтруем заказы

In [98]:
lim_max = info_chek['total_coast'].max()  # тот самый выброс
lim_max

675000.0

In [99]:
normal_orders = info_chek.query('total_coast < @lim_max')['order_id']

и отфильтруем датасет по ним

In [100]:
data = data.query('order_id in @normal_orders')

Теперь построим таблицу с количеством заказов на пользователя и постмотрим есть ли там выбросы.

In [101]:
count_product_on_customer = (data
                             .groupby('customer_id', as_index=False)['order_id']
                             .nunique()
                             .rename(columns=dict(order_id='orders')))


count_product_on_customer.head()

,customer_id,orders
0,000d6849-084e-4d9f-ac03-37174eaf60c4,1
1,001cee7f-0b29-4716-b202-0042213ab038,1
2,00299f34-5385-4d13-9aea-c80b81658e1b,1
3,002d4d3a-4a59-406b-86ec-c3314357e498,1
4,003bbd39-0000-41ff-b7f9-2ddaec152037,1


In [157]:
px.scatter(count_product_on_customer,
                 y='orders',
                 marginal_y='rug',
                 labels={'orders': 'число заказов',
                         'index': 'пользователь'},
                 title='Количество заказов на пользователя')

Видим огоромный выброс в 126 заказов, ликвидируем его .

отфильтруем пользователей по выбранной границе

In [103]:
lim_max = count_product_on_customer['orders'].max()
lim_max

126

In [104]:
normal_customer = (count_product_on_customer
                   .query('orders<@lim_max')['customer_id'])

отфильтруем по пользователям датафрейм

In [105]:
data = data.query('customer_id in @normal_customer')

## Динамика выручки 

Для построения динамики выручки нужна стоимсоть товарной позиции с учетом количества единиц товара, но ее мы посчитали в пункте выше.

Для построения динамики дохода по месяцам нужна еще и дата, но перед этим ее нужно округлить с точностью до месяца:   
например, `2019-12-16 и 2020-10-12` станет `2019-12-01` и `2020-10-01` соответсвенно.

In [106]:

data['date_round'] = (data['date']
                      .apply(lambda x:
                             pd.Timestamp(
                                 pd.Timestamp(
                                     f"{x.month} / {x.year}"
                                 )
                             )
                             .normalize()
                             )
                      )

Проверим себя

In [107]:
data[['date', 'date_round']].iloc[1375:1390]

,date,date_round
1476,2019-01-12,2019-01-01
1477,2019-01-13,2019-01-01
1478,2019-01-13,2019-01-01
1479,2019-01-13,2019-01-01
1480,2019-01-13,2019-01-01
1481,2019-01-13,2019-01-01
1482,2019-01-13,2019-01-01
1483,2019-01-13,2019-01-01
1484,2019-01-13,2019-01-01
1485,2019-01-13,2019-01-01


округление работает корректно как задумывалось.

Построим теперь таблицу с доходами по месяцам

In [108]:
total_price_dynamic = (data
                       .groupby('date_round', as_index=False)['total_price']
                       .sum()
                       .rename(columns=dict(total_price='revenue'))
                       )

total_price_dynamic.head()

,date_round,revenue
0,2018-10-01,391767.0
1,2018-11-01,346199.0
2,2018-12-01,339076.0
3,2019-01-01,254022.0
4,2019-02-01,290355.0


и наконец график

In [109]:
fig = px.line(total_price_dynamic,
              x='date_round',
              y='revenue',
              labels=dict(date_round='дата', revenue='выручка'),
              line_shape='spline',
              title=f"Выручка по месяцам за период с {data['date'].min()} по {data['date'].max()}")

fig.update_traces(line_width=6)

Видно  что в апреле и июне высокая выручка - время готовить рассаду и летом высаживать(магазин же товаров для сада и дома)

## Средний чек и его динамика

Вычислим затем средний чек 

In [110]:
mean_chek = (data
             .groupby('date_round', as_index=False)
             .aggregate(
                 dict(
                     total_price='sum',
                     order_id='nunique'
                 )
             )
             .rename(
                 columns=dict(
                     total_price='revenue',
                     order_id='count_order'
                 )
             )
             )

mean_chek['mean_chek'] = round(mean_chek['revenue'] / mean_chek['count_order'])
mean_chek

,date_round,revenue,count_order,mean_chek
0,2018-10-01,391767.000000,217,1805.0
1,2018-11-01,346199.000000,185,1871.0
2,2018-12-01,339076.000000,237,1431.0
3,2019-01-01,254022.000000,163,1558.0
4,2019-02-01,290355.000000,260,1117.0
5,2019-03-01,290353.000000,236,1230.0
6,2019-04-01,379783.000000,266,1428.0
7,2019-05-01,301504.000000,180,1675.0
8,2019-06-01,407945.000000,160,2550.0
9,2019-07-01,276547.000000,199,1390.0


смотрим через призму времени!

In [111]:
fig = px.line(mean_chek,
              x='date_round',
              y='mean_chek',
              line_shape='spline',
              labels=dict(
                  date_round='дата',
                  mean_chek='средний чек, руб'
              ),
              title=f"Динамика среднего чека по месяцам")

fig.update_traces(line_width=6, line_color='seagreen')

отметим что в июне большой чек, а это значит что покупают на большие суммы но за меньшее число заказов.

## Помесячная динамика средней выручки

Рассмотрим тперь помесячную динамику средней выручки с покупателя.

In [112]:
revenue_month_dynamic = (data
                         .groupby('date_round', as_index=False)
                         .aggregate(dict(total_price='sum',

                                         customer_id='nunique'))
                         .rename(columns=dict(

                             total_price='revenue',
                             customer_id='count_customer')))



revenue_month_dynamic['mean_revenue'] = round(revenue_month_dynamic['revenue'] /
                                              revenue_month_dynamic['count_customer'])



revenue_month_dynamic

,date_round,revenue,count_customer,mean_revenue
0,2018-10-01,391767.000000,178,2201.0
1,2018-11-01,346199.000000,178,1945.0
2,2018-12-01,339076.000000,229,1481.0
3,2019-01-01,254022.000000,152,1671.0
4,2019-02-01,290355.000000,245,1185.0
5,2019-03-01,290353.000000,228,1273.0
6,2019-04-01,379783.000000,237,1602.0
7,2019-05-01,301504.000000,166,1816.0
8,2019-06-01,407945.000000,151,2702.0
9,2019-07-01,276547.000000,187,1479.0


In [113]:
fig = px.line(revenue_month_dynamic,
              x='date_round',
              y='mean_revenue',
              line_shape='spline',
              labels=dict(date_round='дата',
                          mean_revenue='средняя выручка, руб'),
              title=f"Динамика средней выручки с покупателя по месяцам")

fig.update_traces(line_width=6, line_color='lightseagreen')

в Июне 2019 средняя выручка высокая - значит закупаются клиенты которые знакомы с интернет-магазиноом и знают что покупать в этолт месяц .

## Сегментация покупателей на основе истории покупок

Выполним сегментацию пользователей на основе истории их покупок при помощи RFM анализа

Необходимое для этого метода: 
-  давность покупки,    
-  количество покупок,    
- сумма покупок

Для начала созданим таблицу пользователей с первой и последней датой покупки , числом заказов, и стоимостью заказа

In [114]:
table_customer = (data
                  .groupby('customer_id', as_index=False)
                  .aggregate(dict(date='max',

                                  order_id='nunique',

                                  total_price='sum')))


table_customer.rename(columns=dict(date='last_date',
                                   order_id='count_orders',

                                   total_price='revenue'), inplace=True)


table_customer.head()

,customer_id,last_date,count_orders,revenue
0,000d6849-084e-4d9f-ac03-37174eaf60c4,2019-10-16,1,555.0
1,001cee7f-0b29-4716-b202-0042213ab038,2019-02-16,1,442.0
2,00299f34-5385-4d13-9aea-c80b81658e1b,2019-10-14,1,914.0
3,002d4d3a-4a59-406b-86ec-c3314357e498,2019-01-27,1,1649.0
4,003bbd39-0000-41ff-b7f9-2ddaec152037,2019-09-29,1,2324.0


Запишем в данную таблицу сколько дней назад была провденена последняя покупка. 

In [115]:
start_date = data['date'].max()
print('Датой отсчета берем самую последнюю дату -', start_date)

Датой отсчета берем самую последнюю дату - 2020-01-31


In [116]:
table_customer['delta_date'] = (table_customer['last_date']
                                .apply(lambda x:
                                       pd.to_timedelta(start_date - x)
                                       )
                                )


table_customer.head()

,customer_id,last_date,count_orders,revenue,delta_date
0,000d6849-084e-4d9f-ac03-37174eaf60c4,2019-10-16,1,555.0,107 days
1,001cee7f-0b29-4716-b202-0042213ab038,2019-02-16,1,442.0,349 days
2,00299f34-5385-4d13-9aea-c80b81658e1b,2019-10-14,1,914.0,109 days
3,002d4d3a-4a59-406b-86ec-c3314357e498,2019-01-27,1,1649.0,369 days
4,003bbd39-0000-41ff-b7f9-2ddaec152037,2019-09-29,1,2324.0,124 days


In [117]:
table_customer.dtypes

customer_id              object
last_date                object
count_orders              int64
revenue                 float64
delta_date      timedelta64[ns]
dtype: object

Так как данные типа `timedelta64` могут возвращать в перцентилях не дни а секунды то окончательно превратим разницу дат в дни как число для удобных вычислений

In [118]:
table_customer['delta_date'] = table_customer['delta_date'].dt.days
table_customer.dtypes

customer_id      object
last_date        object
count_orders      int64
revenue         float64
delta_date        int64
dtype: object

**Проранжируем клиентов следующим образом**   

Ранжирование по давности  
 - 1- клиенты, покупавшие очень давно  
 - 2 - относительно недавние клиенты
 - 3 - только пришедшие клиенты

 Ранжирование по частоте покупок   
 - 1- клиенты, совершившие очень мало покупок   
 - 2 - клиент, крайне редко покупающий
 - 3 - постоянный покупатель

 Ранжирование по сумме покупки   
 - 1- небольшая сумма покупки  
 - 2 - средние затраты
 - 3 - крупная сумма, потраченная клиентом

Границы значений для рангов определим через перцентили

In [119]:
bounds = [33, 66]

In [120]:

bounds_delta_days = np.percentile(table_customer['delta_date'], bounds)
print(
    f'Границы дней последней покупки: {bounds_delta_days[0]} и {bounds_delta_days[1]}')

bounds_orders = np.percentile(table_customer['count_orders'], bounds)
print(f'Границы количества заказов: {bounds_orders[0]} и {bounds_orders[1]}')


bounds_revenue = np.percentile(table_customer['revenue'], bounds)
print(f'Границы суммы покупки: {bounds_revenue[0]} и {bounds_revenue[1]}')

Границы дней последней покупки: 103.0 и 298.0
Границы количества заказов: 1.0 и 2.0
Границы суммы покупки: 562.0 и 1496.0


Запишем каждый ранг для каждого пользователля в своем столбце

Ранжирование по давности:

In [121]:
table_customer['range_r'] = np.where(
    table_customer['delta_date'] <= bounds_delta_days[0],
    '3',
    np.where(
        table_customer['delta_date'] > bounds_delta_days[1],
        '1',
        '2'
    )
)


table_customer.head()

,customer_id,last_date,count_orders,revenue,delta_date,range_r
0,000d6849-084e-4d9f-ac03-37174eaf60c4,2019-10-16,1,555.0,107,2
1,001cee7f-0b29-4716-b202-0042213ab038,2019-02-16,1,442.0,349,1
2,00299f34-5385-4d13-9aea-c80b81658e1b,2019-10-14,1,914.0,109,2
3,002d4d3a-4a59-406b-86ec-c3314357e498,2019-01-27,1,1649.0,369,1
4,003bbd39-0000-41ff-b7f9-2ddaec152037,2019-09-29,1,2324.0,124,2


Ранжирование по количеству заказов:

In [122]:
table_customer['range_f'] = (np.where(
    table_customer['count_orders'] <= bounds_orders[0],
    '1',
    np.where(table_customer['count_orders'] > bounds_orders[1], '3', '2')))

table_customer.head()

,customer_id,last_date,count_orders,revenue,delta_date,range_r,range_f
0,000d6849-084e-4d9f-ac03-37174eaf60c4,2019-10-16,1,555.0,107,2,1
1,001cee7f-0b29-4716-b202-0042213ab038,2019-02-16,1,442.0,349,1,1
2,00299f34-5385-4d13-9aea-c80b81658e1b,2019-10-14,1,914.0,109,2,1
3,002d4d3a-4a59-406b-86ec-c3314357e498,2019-01-27,1,1649.0,369,1,1
4,003bbd39-0000-41ff-b7f9-2ddaec152037,2019-09-29,1,2324.0,124,2,1


Ранжирование по сумме заказов:

In [123]:
table_customer['range_m'] = (np.where(
    table_customer['revenue'] <= bounds_revenue[0],
    '1',
    np.where(table_customer['revenue'] > bounds_revenue[1], '3', '2')))

table_customer.head()

,customer_id,last_date,count_orders,revenue,delta_date,range_r,range_f,range_m
0,000d6849-084e-4d9f-ac03-37174eaf60c4,2019-10-16,1,555.0,107,2,1,1
1,001cee7f-0b29-4716-b202-0042213ab038,2019-02-16,1,442.0,349,1,1,1
2,00299f34-5385-4d13-9aea-c80b81658e1b,2019-10-14,1,914.0,109,2,1,2
3,002d4d3a-4a59-406b-86ec-c3314357e498,2019-01-27,1,1649.0,369,1,1,3
4,003bbd39-0000-41ff-b7f9-2ddaec152037,2019-09-29,1,2324.0,124,2,1,3


Теперь собирем всю оценку  RFM воедино из соотвествующих столбцов `range_r`, `range_f`, `range_m`

In [124]:
table_customer['group'] = (table_customer[['range_r', 'range_f', 'range_m']]
                           .apply(lambda x:
                                  ''.join(x),
                                  axis=1
                                  )
                           )


table_customer.head()

,customer_id,last_date,count_orders,revenue,delta_date,range_r,range_f,range_m,group
0,000d6849-084e-4d9f-ac03-37174eaf60c4,2019-10-16,1,555.0,107,2,1,1,211
1,001cee7f-0b29-4716-b202-0042213ab038,2019-02-16,1,442.0,349,1,1,1,111
2,00299f34-5385-4d13-9aea-c80b81658e1b,2019-10-14,1,914.0,109,2,1,2,212
3,002d4d3a-4a59-406b-86ec-c3314357e498,2019-01-27,1,1649.0,369,1,1,3,113
4,003bbd39-0000-41ff-b7f9-2ddaec152037,2019-09-29,1,2324.0,124,2,1,3,213


Распределим теперь их по трем большим группам - оценки больше 300 относятся к группе 3 , оценки от 200 до 300 -> 2 группа и остальные в 1 группу. За рязряд сотен отвечает оценка `range_r` - ее и применим для распределния по группам - так быстрее

In [125]:
table_customer['major_group'] = (table_customer['range_r']
                                 .apply(lambda x:
                                        'group_' + x))

table_customer.head()

,customer_id,last_date,count_orders,revenue,delta_date,range_r,range_f,range_m,group,major_group
0,000d6849-084e-4d9f-ac03-37174eaf60c4,2019-10-16,1,555.0,107,2,1,1,211,group_2
1,001cee7f-0b29-4716-b202-0042213ab038,2019-02-16,1,442.0,349,1,1,1,111,group_1
2,00299f34-5385-4d13-9aea-c80b81658e1b,2019-10-14,1,914.0,109,2,1,2,212,group_2
3,002d4d3a-4a59-406b-86ec-c3314357e498,2019-01-27,1,1649.0,369,1,1,3,113,group_1
4,003bbd39-0000-41ff-b7f9-2ddaec152037,2019-09-29,1,2324.0,124,2,1,3,213,group_2


### Число пользователей

Посмотрим на чмсло пользователей в группах RFM

In [126]:
q = (table_customer['group']
     .value_counts()
     .reset_index()
     .sort_values('group', ascending=False))

In [127]:
px.bar(q,
       x='index',
       y='group',
       title='Количество пользователей в группах согласно анализу RFM',
       labels=dict(group='группы', count='количество'))

Как видим, много пользователей с оценкой 323 - постоянные клиеты, 211- недавние клиенты, 111 ,113, 112 - редкие покупатели/ушедшие

Проверим, одинаковые ли группы по числу пользователей

In [128]:
size_group = (table_customer['major_group']
              .value_counts()
              .reset_index()
              .rename(columns=dict(major_group='count', index='group')))

size_group.head()

,group,count
0,group_1,819
1,group_3,798
2,group_2,794


желательно в процентах

In [129]:
size_group['%'] = (size_group['count'] /
                   size_group['count'].sum()*100).round(2)
size_group.head()

,group,count,%
0,group_1,819,33.97
1,group_3,798,33.10
2,group_2,794,32.93


Действительно, почти поровну на 3 части

### Средний чек групп пользователей

In [130]:
customer_with_group = data.merge(table_customer[['customer_id', 'major_group']],
                                 how='left',
                                 on='customer_id')

customer_with_group.head()

,customer_id,order_id,product,quantity,price,datetime,date,total_price,date_round,major_group
0,ee47d746-6d2f-4d3c-9622-c31412542920,68477,комнатное растение в горшке алое вера d12 h30,1,142.0,2018-10-01,2018-10-01,142.0,2018-10-01,group_1
1,ee47d746-6d2f-4d3c-9622-c31412542920,68477,комнатное растение в горшке кофе арабика d12 h25,1,194.0,2018-10-01,2018-10-01,194.0,2018-10-01,group_1
2,ee47d746-6d2f-4d3c-9622-c31412542920,68477,радермахера d12 см h20 см,1,112.0,2018-10-01,2018-10-01,112.0,2018-10-01,group_1
3,ee47d746-6d2f-4d3c-9622-c31412542920,68477,хризолидокарпус лутесценс d9 см,1,179.0,2018-10-01,2018-10-01,179.0,2018-10-01,group_1
4,ee47d746-6d2f-4d3c-9622-c31412542920,68477,циперус зумула d12 см h25 см,1,112.0,2018-10-01,2018-10-01,112.0,2018-10-01,group_1


In [131]:
mean_chek = (customer_with_group
             .groupby(['date_round', 'major_group'], as_index=False)
             .aggregate(
                 dict(
                     total_price='sum',
                     order_id='nunique'
                 )
             )
             )


mean_chek['mean_chek'] = (mean_chek['total_price'] /
                          mean_chek['order_id']
                          ).round()

mean_chek.head()

,date_round,major_group,total_price,order_id,mean_chek
0,2018-10-01,group_1,335799.0,170,1975.0
1,2018-10-01,group_3,55968.0,47,1191.0
2,2018-11-01,group_1,290965.0,139,2093.0
3,2018-11-01,group_3,55234.0,46,1201.0
4,2018-12-01,group_1,247828.0,158,1569.0


In [132]:
fig = px.line(mean_chek,
              x='date_round',
              y='mean_chek',
              color='major_group',
              title='Средний чек групп покупателей',
              labels=dict(date_round='дата', mean_chek='средний чек',
                          major_group='группа покупателей'),
              line_shape='spline'
              )

fig.update_traces(line_width=4)

Как видим, группа 1 (ушедщие пользователи) имеют средний чек выше ,чем у постоянных пользователей(группа 3) с октября по апрель - покупают на большие суммы за меньшее число заказов. А в группе 2 (неактивные пользователи) просаживается средний чек в феврал и марте - то есть они совершают много заказов. У постоянных пользователей средний чек скачет в марте, июне , сентябре и декабре - совершают дорогие покупки за меньшее число заказов.

### Товары

Совершим теперь категоризацию товаров обобщив по принадлежности, например, если есть слова `рассада` или `сорт` то ему присвоится `рассада и растения`. Напишем функцию с сключевыми словами и окнчаниями слов, на основе котрых можно присвоить категорию. 

In [133]:
def categorizator(words):
    """категоризатор товаров"""

    # проверка на входящие слова - если есть то вернет Правду
    def proverka(key_words):
        
        return any(
            filter(
                lambda x: x in words,
                key_words
            )
        )

    if proverka(['рассада', 'растение', 'цветок', 'в упаковке',
                 'пеларгония', 'бакопа', 'вербена',
                 'сорт', 'герань', 'фуксия', 'лист',
                 'петуния', 'черенок', 'в горшке', 'баклажан',
                 'калибрахоа', 'цикламен', 'эвкалипт', 'колокольчик',
                 'циперус', 'раннеспел', 'крупноцветков',
                 'среднеспел', 'скороспел', 'комнатн', 'олистн', 'лаванда', 'кипарис',
                 'томат', 'гвоздика', 'дыня', 'фиалка', 'смесь', 'в кассете', 'лекарствен', 'обыкновен',

                 'ум ', 'ения ', 'егия ', 'ус ', 'ния ', 'ония ', 'ис ', 'ия ', 'ея ', 'ин ', 'лик ', 'лла ',
                 'ола ', 'ера ', 'оза ', 'барвинок', 'сантолина',

                 'базилик', 'травянист', 'тимьян', 'зверобой ', 'мята', 'каланхое', 'замиокулькас ',
                 'камнеломка', 'овсянница', 'горшок', 'календула', 'юкка', 'солидаго ', 'подсолнечник',
                 'роза', 'котовник', 'афеляндра ', 'хризантема', 'лавр', 'муррайя ', 'огурец ',
                 'вербейник ', 'лобелия', ' микс ', 'арбуз', 'морковь', 'алоэ', 'бархатцы', 'монарда',
                 'настурция', 'садов', 'капуста', 'седум', 'флокс ', 'крассула ', 'тюльпан ',
                 'калатея ', 'мирт ', 'ясколка ', 'кофе', 'настурция', 'чабер', 'петрушка ', 'земляника ',
                 'эхеверия ', 'незабудка ', 'мединилла ', 'нолина ', 'папоротник', 'лен ', 'львиный зев', 'георгина ',
                 'клубника ', 'каллуна ', 'смолевка ', 'драцена ', 'пахира ', 'анемона ', 'лантана ']):

        return 'рассада и растения'

    elif proverka(['муляж', 'новогод', 'кашпо', 'искусствен', 'декор']):
        return 'декор'

    elif proverka(['для ванн', 'мыл', 'для туалет', 'для унитаза', 'в ванну']):
        return 'предметы для ванной комнаты'

    elif proverka(['средство', 'чист', 'для мытья', 'антижир', 'для стирки', 'сумка', 'тележка', 'вешал', 'для посудомоеч',
                   'сушилка', 'стремян', 'лестница', 'для хранения', 'окон', 'крючок', 'кофр ', 'корзин',
                   'плечики', 'щетка', 'подкладка', 'для глаженного', 'короб', 'веник', 'для тележки', 'прищеп',
                   'для белья', 'одежды', 'гладил', 'корыто', 'подрукав', 'ковш', 'для штор', 'бидон', 'мусор', 'лапчатка ',
                   'таз', 'ванна ', 'вантуз', 'для одежды', 'для досок', 'ведр', 'урна', 'зажигал', 'рассекатель ',
                   'окномойка', 'швабра', 'подголовник', 'котел', 'для костюма', 'для утюг', 'совок', 'совком', 'ерш', 'ящик', 'ложка обувная',
                   'тряпкодержатель', 'перчатки', 'для полировки', 'зубная паста', 'eurohouse', 'rozenbal', 'стирки']):
        return 'хозтовары'

    elif proverka(['полки', 'комод', 'для обуви',
                   'обувница', 'пуф ', 'придверный',
                   'этажерка', 'коврик', 'картина', 'золот', 'фоторамка']):
        return 'интерьер, мебель'

    elif proverka(['махров', 'ткань', 'флис',
                   'пух', 'покрывало', 'микрофибр', 'чехол ',
                   'скатерть', 'халат', 'хлопок',
                   'постельн', 'полиэстер']):
        return 'текстиль'

    elif proverka(['блюд', 'кувшин', 'салатник', 'для кухни', 'кухонн',
                   'кружка', 'сковород', 'хлебница', 'сито ', 'webber',
                   'салфет', 'фарфор', 'термос ', ' эмал', 'банк', 'кастр', 'чеснок',
                   'тарелка', 'банка', 'терка', 'миксер', 'чайн', 'варка', 'толкушка', 'противень ',
                   'миска', 'для муки', 'тортница', 'штопор', 'венчик', 'кондитер', 'отделитель ',
                   'скалка', 'столов', 'для выпечки', 'pasabahce', 'лоток ', 'attribute blossom',
                   'для свч', 'контейнер', 'luminarc', 'стакан', 'pyrex', 'tvs', 'viva attribute',
                   'разделочн', 'кекс', 'картофелемялка', 'bamboo', 'wilmax']):

        return 'посуда, товары для кухни'

    elif proverka(['термометр', 'нож', 'штангенциркуль', 'весы', 'сверло', 'сверел',
                   'шило',
                   'фал', 'шпагат ', 'шнур ', 'фиксатор',
                   'линейка', 'напильник', 'инструмент', 'измерительн']):
        return 'инструменты, такелаж'

    elif proverka([' вт ', ' квт ', 'электр', 'perfectolight']):
        return 'электроприборы'

    elif proverka(['петля', 'стяжка', 'шпингалет', 'крепеж', 'завертка', 'ручка']):
        return 'фурнитура'

    elif proverka(['подароч']):
        return 'подарки на праздники'

    else:
        return words  # проверка

применим и проверим

In [134]:
customer_with_group['category_product'] = (customer_with_group['product']
                                           .apply(categorizator))


customer_with_group['category_product'].value_counts()

рассада и растения             4587
хозтовары                      1581
посуда, товары для кухни        392
декор                           196
предметы для ванной комнаты     169
интерьер, мебель                145
текстиль                         73
инструменты, такелаж             73
фурнитура                        17
электроприборы                    4
подарки на праздники              3
Name: category_product, dtype: int64

Товары побеждены и категоризированы

In [135]:
pop_category = (customer_with_group
                .groupby(['major_group', 'category_product'], as_index=False)
                .agg({'quantity': 'sum'}))

перед построением гррафика отсортируем по группе по числу товаров

In [136]:
pop_category = (pop_category
                .sort_values(['major_group', 'quantity'], ascending=[True, False]))

In [137]:
px.bar(pop_category,
       x='major_group',
       y='quantity',
       color='category_product',
       barmode='group',
       color_discrete_sequence=px.colors.qualitative.Prism,
       labels=dict(quantity='кол-во купленных товаров в категории',
                   major_group='сегмент пользователей',
                   category_product='категория товара'),
       title='Популярные категории товаров сегментов пользователей',
       )

Для группы 1 (ушедшие пользователи) характерны товары: рассада, хозтовары, посуда и товары для кухни, декор, фурнитура.  
Для группы 2 (неактивные пользователи) характерны покупки в виде рассады и расстения и хозтовары.  
Для группы 3 (постоянные покупатели) характерны покупки хозтоваров и также рассады и растений (но меньше чем в других сегиентах).

## Анализ сезонности товаров в разрезе сегментов покупателей

Построение графика произведем по месяцам - пригодится округленная дата.

In [138]:
history_category = (customer_with_group

                    .groupby(['major_group',
                              'date_round',
                              'category_product'],
                             as_index=False)['quantity']
                    .sum())


history_category.head()

,major_group,date_round,category_product,quantity
0,group_1,2018-10-01,декор,122
1,group_1,2018-10-01,"инструменты, такелаж",7
2,group_1,2018-10-01,"интерьер, мебель",17
3,group_1,2018-10-01,"посуда, товары для кухни",74
4,group_1,2018-10-01,предметы для ванной комнаты,15


In [152]:
fig = px.line(history_category.query('major_group=="group_1"'),
              x='date_round',
              y='quantity',

              color='category_product',
              title='Сегмент 1й. Покупка товаров разных категорий по месяцам',
              labels=dict(quantity='кол-во проданного товара',
                          category_product='категория товара',
                          date_round='месяц'),
              color_discrete_sequence=px.colors.qualitative.Vivid,
              template='simple_white')

fig.update_traces(
    mode='markers+lines',
      line_shape='spline',
      line_width=3
)

Видим, что первый сегмент активно закупался рассадой и растениями в октябре,феврале и марте. Также приобретал товары для декора - много в ноябре и феврале.Фурнитура больше приобретается в октябре и декабре. Посуда и товары для кухни чаще приобретаются в декабре.  

In [153]:
fig = px.line(history_category.query('major_group=="group_2"'),
              x='date_round',
              y='quantity',

              color='category_product',
              title='Сегмент 2й. Покупка товаров разных категорий по месяцам',
              labels=dict(quantity='кол-во проданного товара',
                          category_product='категория товара',
                          date_round='месяц'),
              color_discrete_sequence=px.colors.qualitative.Vivid,
              template='simple_white'
              )

fig.update_traces(mode='markers+lines',
                  line_shape='spline',
                  line_width=3
                  )

Сегмент 2(неактивные пользователи). Активно закупаются рассадой и растениями особенно в апреле и июнь. В мае заказывали для декора больше чем за предстиавленный период. В июне приобретают больше хозтоваров.

In [154]:
fig = px.line(history_category.query('major_group=="group_3"'),
              x='date_round',
              y='quantity',

              color='category_product',
              title='Сегмент 3й. Покупка товаров разных категорий по месяцам',
              labels=dict(quantity='кол-во проданного товара',
                          category_product='категория товара',
                          date_round='месяц'),
              color_discrete_sequence=px.colors.qualitative.Vivid,
              template='simple_white')

fig.update_traces(mode='markers+lines',
                  line_shape='spline',
                  line_width=3
                  )

Сегмент 3 (постоянные покупатели). Активно приобретают растения и хозтовары в октябре-декабе 2019 и январе 2020.   
В остальные месяцы менее активны,но продолжают приобретать хозтовары. В Октябре 2019 Активно приобретают фурнитуру и товары для декора.

# Проверка гипотез

## Гипотеза 1. Средние чеки сегментов групп равны

Сформулируем гипотезу   
1) Нулевая гипотеза: средние чеки сегментов групп равны   
    Альт. гипотеза: средние чеки сегментов групп разные

In [142]:
hipotesys_1 = dict(H_0='средние чеки сегментов групп равны',
                   H_1='средние чеки сегментов групп разные')

И посчитаем средний чек для каждого пользователя

In [143]:
mean_chek_groups = (customer_with_group
                    .groupby(['major_group', 'customer_id'], as_index=False)
                    .aggregate(
                        {'total_price': 'sum', 'order_id': 'nunique'})
                    .rename(columns=dict(order_id='orders')))

mean_chek_groups.head()

,major_group,customer_id,total_price,orders
0,group_1,001cee7f-0b29-4716-b202-0042213ab038,442.0,1
1,group_1,002d4d3a-4a59-406b-86ec-c3314357e498,1649.0,1
2,group_1,004d24e9-4a6c-4d0e-8727-8391dfd4b43a,1536.0,1
3,group_1,00bd74b1-2792-47db-a2f1-680a09ac5026,937.0,1
4,group_1,018dc738-2846-464d-a421-126a8ed64bc5,940.0,1


In [144]:
mean_chek_groups['mean_chek'] = (mean_chek_groups['total_price'] / 
                                 mean_chek_groups['orders'])
mean_chek_groups.head()

,major_group,customer_id,total_price,orders,mean_chek
0,group_1,001cee7f-0b29-4716-b202-0042213ab038,442.0,1,442.0
1,group_1,002d4d3a-4a59-406b-86ec-c3314357e498,1649.0,1,1649.0
2,group_1,004d24e9-4a6c-4d0e-8727-8391dfd4b43a,1536.0,1,1536.0
3,group_1,00bd74b1-2792-47db-a2f1-680a09ac5026,937.0,1,937.0
4,group_1,018dc738-2846-464d-a421-126a8ed64bc5,940.0,1,940.0


настало время статистического теста. Сравнивать будем группы между собой попарно . Использовать будем t-тест для независимых выборок. 

Затем напишем функцию выводящую результат теста

In [145]:
def get_result_ttest(list_groups, hipotesys, table, column):
    """Проводит t-тест между парами групп пользователей. Возвращает принятую гипотезу.

    `list_groups` - список групп,\n
    `hipotesys` - гипотеза,\n
    `table` - таблица на основе которой проводится тест,\n
    `columns` - колонка в которой данные для теста"""

    first_group = table.query('major_group==@list_groups[0]')[column]
    second_group = table.query('major_group==@list_groups[1]')[column]

    p_value = st.ttest_ind(first_group, second_group)[1]

    # поправка Бонферони, где "3" - количество тестов
    alpha = 0.05
    alpha_corr = alpha / 3

    print('\nТест между группами', list_groups)

    print(f'Среднее {list_groups[0]}:', round(first_group.mean()))
    print(f'Среднее {list_groups[1]}:', round(second_group.mean()))

    print('p_value:', '{:.3}'.format(p_value))

    if p_value > alpha_corr:
        print(
            f'Результаты теста не противоречат нулевой гипотезе: {hipotesys.get("H_0")}')
    else:
        print(
            f'Отвергаем нул. гипотезу в пользу альтернативной: {hipotesys.get("H_1")}')

Созданим кортеж пар груп между которыми будет проведен тест

In [146]:
list_pair_group = (('group_1', 'group_2'),
                   ('group_1', 'group_3'),
                   ('group_2', 'group_3'))

In [147]:
for i in list_pair_group:
    get_result_ttest(i,
                     hipotesys_1,
                     table=mean_chek_groups,
                     column='mean_chek')


Тест между группами ('group_1', 'group_2')
Среднее group_1: 1689
Среднее group_2: 1660
p_value: 0.89
Результаты теста не противоречат нулевой гипотезе: средние чеки сегментов групп равны

Тест между группами ('group_1', 'group_3')
Среднее group_1: 1689
Среднее group_3: 962
p_value: 5.43e-07
Отвергаем нул. гипотезу в пользу альтернативной: средние чеки сегментов групп разные

Тест между группами ('group_2', 'group_3')
Среднее group_2: 1660
Среднее group_3: 962
p_value: 0.000418
Отвергаем нул. гипотезу в пользу альтернативной: средние чеки сегментов групп разные


Согласно теста, средние чеки    
сегмента 1 и сегмента 2 равны

## Гипотеза 2. Средняя выручка сегментов групп равны

Сформулируем гипотезы 

In [148]:
hipotesys_2 = dict(H_0='средние выручки сегментов групп равны',
                   H_1='средние выручки сегментов групп разные')

Далее приготовим данные - найдем среднюю выручку для каждого пользователя

In [149]:
mean_revenue_month = (customer_with_group
                      .groupby(['major_group', 'customer_id'], as_index=False)
                      .aggregate(dict(total_price='sum', quantity='sum'))
                      .rename(columns=dict(total_price='revenue', quantity='count_customer')))


mean_revenue_month['mean_revenue'] = round(mean_revenue_month['revenue'] /
                                           mean_revenue_month['count_customer'])


mean_revenue_month.head()

,major_group,customer_id,revenue,count_customer,mean_revenue
0,group_1,001cee7f-0b29-4716-b202-0042213ab038,442.0,1,442.0
1,group_1,002d4d3a-4a59-406b-86ec-c3314357e498,1649.0,1,1649.0
2,group_1,004d24e9-4a6c-4d0e-8727-8391dfd4b43a,1536.0,12,128.0
3,group_1,00bd74b1-2792-47db-a2f1-680a09ac5026,937.0,1,937.0
4,group_1,018dc738-2846-464d-a421-126a8ed64bc5,940.0,10,94.0


Проведем тесты

In [150]:
for i in list_pair_group:
    get_result_ttest(i,
                     hipotesys_2,
                     table=mean_revenue_month,
                     column='mean_revenue')


Тест между группами ('group_1', 'group_2')
Среднее group_1: 650
Среднее group_2: 664
p_value: 0.79
Результаты теста не противоречат нулевой гипотезе: средние выручки сегментов групп равны

Тест между группами ('group_1', 'group_3')
Среднее group_1: 650
Среднее group_3: 818
p_value: 0.000303
Отвергаем нул. гипотезу в пользу альтернативной: средние выручки сегментов групп разные

Тест между группами ('group_2', 'group_3')
Среднее group_2: 664
Среднее group_3: 818
p_value: 0.00347
Отвергаем нул. гипотезу в пользу альтернативной: средние выручки сегментов групп разные


Согласно теста, средние выручки   
сегмента 2 и сегмента 3 а также   
сегмента 1 и сегмента 3   
разные.

# Общий вывод

Декомпозировав, и совершив предобработку данных, вкдючающую в себя:  
- проверку на пропуски,  
- проверку типов данных, 
- форматирование названий столбцов, 
- проверка на явные и неявные дубликаты,  
- устранение неявных дубликатов,  
- приведение даты к нужному формату и виду.

Затем произвели исследовательский анализ данных и выяснили, 
- за какой период взяты двнные  
- изучили данные на выбросы и устранили их  
- Изучили динамику выручки, среднего чека и средней выручки  
- Сегментировали пользователей на основе исории покупок при помощи RFM анализа   
- Изучили средний чек сегментов пользователей.  
- Изучили какие категории товаров покупаются в сегментах  
- Произвыели анализ сезонности товаров в разрезе сегментов покупателей и проверили гипотезы о средних чеках сегментов и средних выручках сегментов.  

Обнаружили и нашли, что: 
 - данные взяты за период 2018-10-01 - 2020-01-31   
 - Проверили данные на выбросы и устранили их во избежание искажения данных  
 - Построили динамику выручки за предоставленный период  
и выяснили  , что:   
в Июне 2019 средняя выручка высокая - значит закупаются клиенты которые знакомы с интернет-магазиноом и знают что покупать в этолт месяц .
- При помощи RFM анализа выяснили что много  постоянных клиеты, недавних клиенты, редких покупателей/ушедших.   

 - группа 1 (ушедщие пользователи) имеют средний чек выше ,чем у постоянных пользователей(группа 3) с октября по апрель - покупают на большие суммы за меньшее число заказов. А в группе 2 (неактивные пользователи) просаживается средний чек в марте - то есть они совершают много заказов. У постоянных пользователей средний чек скачет в марте, июне , сентябре и декабре - совершают дорогие покупки за меньшее число заказов.

-Для группы 1 (ушедшие пользователи) характерны товары: рассада, хозтовары, посуда и товары для кухни, декор.  
Для группы 2 (неактивные пользователи) характерны покупки в виде рассады и расстения и хозтовары.  
Для группы 3 (постоянные покупатели) характерны покупки хозтоваров и также рассады и растений (но меньше чем в других сегиентах).
  
- первый сегмент активно закупался рассадой и растениями в октябре,феврале и марте. Также приобретал товары для декора - много в ноябре и феврале.Фурнитура больше приобретается в октябре и декабре. Посуда и товары для кухни чаще приобретаются в декабре.  

- Сегмент 2(неактивные пользователи). Активно закупаются рассадой и растениями особенно в апреле и июнь. В мае заказывали для декора больше чем за предстиавленный период. В июне приобретают больше хозтоваров.

- Сегмент 3 (постоянные покупатели). Активно приобретают растения и хозтовары в октябре-декабе и январе.   
В остальные месяцы менее активны,но продолжают приобретать хозтовары. В Октябре Активно приобретают фурнитуру и товары для декора.


- выяснили что средние чеки ушедших клиентов и неактивных клиентов равны   
- средние выручки ушедших пользователей и неактивных пользователей равны. 






Рекомендации:   
отправлять рассылки рекламных предложениях для разных сегментов по месяцам о категориях товаров.    
Ниже приведены месяца и категории товаров для каждого сегмента.   


Ушедшие (Сегмент 1)  
Январь - рассылать о рассаде, хозтоварах   
Февраль - рассада,хозтовары, декор   
март - рассада, хозтовапры   
апрель - рассада и хозтовары    
Октябрь - рассада, хозтовары, фурнитура  
Ноябрь  - декор, хозтовары, рассада   
Декабрь - хозтовары, фурнитура, товары для кухни     




Неактивные ( Сегмент 2)  

апрель - рассада    
май - рассада + декор   
июнь - рассада и хозтовары   
июль - рассада и хозтовары   
август - рассада и хозтовары,   
сентябрь - товары для кухни  
октябрь - рассада  


Постоянные пользователи (Сегмент 3)    
Январь - рассада и хозтовары, товары для кухни   
Февраль - рассада и хозтовары   
Март - рассада и хозтовары   
Апрель - рассада и хозтовары   
Май - хозтовары   
Июнь  - хозтовары   
Июль - хозтовары   
Август - хозтовары   
Сентябрь - хозтовары и рассада   

Октябрь - хозтовары, рассада, фурнитура   
Ноябрь - рассада, хозтовары, товары для кухни     
Декабрь - хозтовары и рассада + товары для кухни        